# Generate index weights using dedicated service - C#

### Overview
Reference data encompasses a wide range of specification data about:
* financial instrument such as asset class, symbol, maturity, etc.
* counterparties such as issuer information.
* pricing such as Open, high low and close prices.

The challenge with reference data is that it tends to be sourced from multiple sources: internal, counterparties and providers.

Reference data dedicated service is a result of various data sources and fields sourcing, validation, cross-checking and normalization.<br>

This sample requests **on-demand** exchange data and enables fields mapping by calling a dedicated static data service, making available clean data.

### Inputs/outputs
Index weights sample is designed to help you searching and requesting index components reference data by using a wide range of commonly used fields.

It takes an index code as per input filter and returns index components and weights.

### Services used
This sample uses *gRPC requests* in order to retrieve index components and weights from the hosted service. The queried endpoint in this script are:
* *ComponentsService*: to directly retrieve index data objects from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Indices.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*

***

# Run Index weights sample

### Step 1: Install packages

In [ ]:
#r "nuget: Systemathics.Apis"
#r "nuget: CsvHelper"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Indices.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Import/export csv files
using CsvHelper;
using System.IO;
using System.Globalization;
// Plot graphs
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
//var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

// Define API entry
var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Request creation

The following code snippet enables to select the **index** by its *Name*:

In [ ]:
// Set index name
var index = "nasdaq 100";

The following code snippets call the service, generate the request and return the reply: 

In [ ]:
// Instantiate the indice components service
var service = new ComponentsService.ComponentsServiceClient(channel);

// Generate the indice components request
var request = new ComponentsRequest { Identifier = index };

// Process the indice components response
var reply = service.Components(request, headers);

// Display the results
display(reply.Components)

### Step 4: Retrieve index data

#### 4.1 Export index data

The following code snippet exports index data to a *csv file*:

In [ ]:
using (var writer = new StreamWriter($"{index}.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    csv.WriteRecords(reply.Components);
}

#### 4.2 Visualize index data

The following code snippet displays index components and weights. <br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:
var data = reply.Components.Select(elt => new Tuple<String, double>(elt.Identifier.Ticker, elt.Weight));

In [ ]:
var chart = Chart.Pie(data);
chart.WithTitle($"{index} components and weights");
chart.Width = 1500;
chart.Height = 800;
display(chart);